<a href="https://colab.research.google.com/github/rsrini7/Colabs/blob/main/PydanticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pydantic-ai langchain-community sentence-transformers scikit-learn numpy --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [2]:
# pydantic_instructor_rag_simulation_openrouter_litellm_v1.py
from google.colab import userdata # Make sure this is at the top of your file
import os
# ... other imports ...

def main():
    print("--- Starting PydanticAI/Instructor (v1.x.x) RAG Simulation with OpenRouter via LiteLLM ---")

    # SET THE ENVIRONMENT VARIABLE HERE
    try:
        openrouter_api_key_value = userdata.get('OPENROUTER_API_KEY')
        os.environ['OPENROUTER_API_KEY'] = openrouter_api_key_value
        print("OpenRouter API Key loaded from Colab Secrets and set in os.environ.")
    except userdata.SecretNotFoundError:
        print("ERROR: OPENROUTER_API_KEY not found in Colab Secrets. Please add it.")
        return # Stop execution if key is not found
    except Exception as e:
        print(f"ERROR: Could not load or set OpenRouter API Key: {e}")
        return # Stop execution on other errors

    # ... rest of your main() function ...
    # (Simulate Data Retrieval, Generate Structured Answer, etc.)
    # LiteLLM, when called by instructor or the openai client, will now check
    # os.environ['OPENROUTER_API_KEY'] for "openrouter/..." models.

In [ ]:
# !litellm --model openrouter/openai/gpt-3.5-turbo --api_base https://openrouter.ai/api/v1 --port 8000

In [1]:
# !litellm --model openrouter/openai/gpt-3.5-turbo --port 8000&

In [10]:
!pip install nest-asyncio

In [17]:
# pydantic_ai_rag_openrouter_PYPI_VERSION.py

from google.colab import userdata
import os
import logging
import sys
from typing import List, Optional


from openai import AsyncOpenAI

# --- Pydantic-AI (from PyPI: pip install pydantic-ai) ---
from pydantic import BaseModel, Field
from pydantic_ai import Agent # The main class from the library
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

# --- For Document Handling & Embeddings (Simplified) ---
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

import nest_asyncio

nest_asyncio.apply()

# --- Configuration & Constants ---
EMBED_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
DATA_DIR = "./data_pydantic_ai_pypi"
SAMPLE_FILE_NAME = "sample_pydantic_ai_pypi.txt"
# For pydantic-ai's OpenAIModel, we need to pass the model string OpenRouter expects
OPENROUTER_MODEL_NAME_FOR_CLIENT = "openai/gpt-3.5-turbo"

# --- 1. Define Pydantic Model for Structured LLM Output ---
class RAGAnswerPyPI(BaseModel):
    answer: str = Field(..., description="The concise answer to the user's question, based *only* on the provided context.")
    context_was_sufficient: bool = Field(..., description="True if the provided context was sufficient to answer the question, False otherwise.")
    # The pydantic-ai library might be simpler, let's keep the model less complex initially
    # supporting_facts: Optional[List[str]] = Field(default=None, description="A list of key facts supporting the answer.")

# --- Main Script Logic ---
def main():
    print("--- Starting RAG with 'pydantic-ai' (PyPI version) & OpenRouter ---")

    # 0. Setup: API Keys and Sample Data
    try:
        openrouter_api_key = userdata.get('OPENROUTER_API_KEY')
        # pydantic-ai's OpenAIModel will need the API key.
        # It might also need OPENAI_API_KEY env var or accept it directly.
        print("OpenRouter API Key loaded from Colab Secrets.")
    except userdata.SecretNotFoundError:
        print("ERROR: OPENROUTER_API_KEY not found in Colab Secrets. Please add it.")
        sys.exit(1)
    except Exception as e:
        print(f"ERROR: Could not load OpenRouter API Key: {e}")
        sys.exit(1)

    sample_file_path = os.path.join(DATA_DIR, SAMPLE_FILE_NAME)
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)
    if not os.path.exists(sample_file_path):
        with open(sample_file_path, "w") as f:
            f.write("""The 'pydantic-ai' library from PyPI aims to provide structured outputs from LLMs.
It uses Pydantic models to define the schema. An AI class with a configured model (e.g., OpenAIModel) is used.
This example attempts to use it for a RAG task.
Context is retrieved separately and then passed to the LLM via pydantic-ai for structured generation.
OpenRouter can be used if the underlying OpenAI client used by pydantic-ai can be configured.
AI developments include areas like machine learning and natural language processing.
""")
        print(f"Created dummy sample file: '{sample_file_path}'")

    # --- Simplified RAG: Document Loading, Chunking, Embedding ---
    print("\n--- 1. Preparing Data (Load, Chunk, Embed) ---")
    try:
        loader = TextLoader(sample_file_path)
        documents = loader.load()
        if not documents:
            print(f"Warning: No documents loaded from '{sample_file_path}'.")
            sys.exit(1)
        print(f"Loaded {len(documents)} document(s).")

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=30)
        chunks = text_splitter.split_documents(documents)
        chunk_texts = [chunk.page_content for chunk in chunks]
        print(f"Split into {len(chunk_texts)} chunks.")

        print(f"Loading embedding model: '{EMBED_MODEL_NAME}'")
        embed_model = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)
        chunk_embeddings = np.array(embed_model.embed_documents(chunk_texts))
        print(f"Embedded {len(chunk_embeddings)} chunks. Embedding dim: {chunk_embeddings.shape[1]}")
    except Exception as e:
        print(f"ERROR during data preparation or embedding: {e}")
        sys.exit(1)

    # --- Simplified RAG: Retrieval ---
    print("\n--- 2. Performing Retrieval ---")
    query = "How does the pydantic-ai library work?"
    try:
        query_embedding = np.array(embed_model.embed_query(query)).reshape(1, -1)
        similarities = cosine_similarity(query_embedding, chunk_embeddings)[0]
        top_k = 2
        retrieved_indices = np.argsort(similarities)[-top_k:][::-1]

        retrieved_context_texts = [chunk_texts[i] for i in retrieved_indices]
        retrieved_context_combined = "\n---\n".join(retrieved_context_texts)

        print(f"Query: '{query}'")
        print(f"Retrieved {len(retrieved_context_texts)} context chunk(s):")
        for i, idx in enumerate(retrieved_indices):
            print(f"  Context {i+1} (Similarity: {similarities[idx]:.4f}): {chunk_texts[idx][:150].strip()}...")
    except Exception as e:
        print(f"ERROR during retrieval: {e}")
        sys.exit(1)


    # --- 3. Structured Generation with 'pydantic-ai' (PyPI version) & OpenRouter ---
    print("\n--- 3. Generating Structured Answer with 'pydantic-ai' (PyPI) ---")
    try:
        # Configure the AI model for pydantic-ai
        # We need to pass api_key and base_url to OpenAIModel if it supports it,
        # or ensure the underlying openai client it uses is configured.
        # Based on the (limited) pydantic-ai source, OpenAIModel takes 'api_key'
        # and might use the standard 'OPENAI_API_BASE' env var or allow passing 'base_url'.

        # Attempt 1: Pass directly to OpenAIModel
        # This is a guess based on common patterns; pydantic-ai's docs are sparse here.

        # Fallback: Set environment variable for OpenAI client if pydantic-ai uses it implicitly
        os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

        llm_model_config = OpenAIModel(
            model_name=OPENROUTER_MODEL_NAME_FOR_CLIENT,
            provider=OpenAIProvider(api_key=openrouter_api_key,
                    base_url="https://openrouter.ai/api/v1",)
        )

        ai_instance = Agent(
            model=llm_model_config,
            response_model=RAGAnswerPyPI # The Pydantic model for the output
        )

        prompt_for_pydantic_ai = f"""
Based *only* on the following context, answer the user's question.
If the context is insufficient, reflect that in your answer.

Context:
---
{retrieved_context_combined if retrieved_context_combined else "No relevant context was found."}
---

User Question: {query}
"""
        print(f"Using model for generation via pydantic-ai: {OPENROUTER_MODEL_NAME_FOR_CLIENT}")

        # Generate the structured response
        # The generate method takes the prompt (which includes our context and query)
        structured_answer_pypi = ai_instance.run_sync(prompt_for_pydantic_ai)


        print("\nStructured LLM Response (from pydantic-ai PyPI):")
        if structured_answer_pypi:
            print(f"  Answer: {structured_answer_pypi.output}")
            print(f"  Usage: {structured_answer_pypi.usage()}")
        else:
            print("  pydantic-ai did not return a structured answer.")

    except Exception as e:
        print(f"ERROR during structured answer generation with 'pydantic-ai' (PyPI): {e}")
        import traceback
        traceback.print_exc()

    print("\n--- RAG with 'pydantic-ai' (PyPI version) Finished ---")

if __name__ == "__main__":
    main()

--- Starting RAG with 'pydantic-ai' (PyPI version) & OpenRouter ---
OpenRouter API Key loaded from Colab Secrets.

--- 1. Preparing Data (Load, Chunk, Embed) ---
Loaded 1 document(s).
Split into 3 chunks.
Loading embedding model: 'sentence-transformers/all-MiniLM-L6-v2'
Embedded 3 chunks. Embedding dim: 384

--- 2. Performing Retrieval ---
Query: 'How does the pydantic-ai library work?'
Retrieved 2 context chunk(s):
  Context 1 (Similarity: 0.7035): The 'pydantic-ai' library from PyPI aims to provide structured outputs from LLMs.
It uses Pydantic models to define the schema. An AI class with a con...
  Context 2 (Similarity: 0.4902): Context is retrieved separately and then passed to the LLM via pydantic-ai for structured generation.
OpenRouter can be used if the underlying OpenAI...

--- 3. Generating Structured Answer with 'pydantic-ai' (PyPI) ---
Using model for generation via pydantic-ai: openai/gpt-3.5-turbo

Structured LLM Response (from pydantic-ai PyPI):
  Answer: The pydantic-